In [1]:
#Import Necessary Libraries
%load_ext autoreload
%autoreload 2
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pylab as pl
from scipy.stats import zscore, sem
import datetime as dt
import utils
import fiona 

from shapely.geometry import Point
from shapely.geometry import shape
from shapely.geometry import Polygon, LineString
from shapely import wkt
from tqdm.notebook import tqdm_notebook

In [2]:
#Load Variables:
start_year = 2017
end_year = 2021
mapbox_accesstoken = None

# Load NASA LANDSAT Temperature Data

In [4]:
#Load the .geo column as a json
heat_all_landsats = pd.DataFrame(columns=['Year', 'ST_B10', 'Long', 'Lat'],dtype="float")
for year in tqdm_notebook(np.arange(start_year, end_year+1)):
    year_landsat_df = pd.read_csv(r'data\NYC_Landsat_HeatData_062722/NYC_CD_Heat_{}_per_pixel_from_5_to_9.csv'.format(year))
    latitudes = [json.loads(pixel)['coordinates'][1] for pixel in year_landsat_df['.geo']]
    longitudes = [json.loads(pixel)['coordinates'][0] for pixel in year_landsat_df['.geo']]

year_landsat_df['Long'] = longitudes
year_landsat_df['Lat']  = latitudes
year_landsat_df['Year'] = year
year_landsat_df = year_landsat_df.rename({'ST_B10_{}'.format(year):'ST_B10'}, axis=1)

#Apprend all the new info. to the df. 
heat_all_landsats = heat_all_landsats.append(year_landsat_df[['Year','ST_B10','Long','Lat']])

#Average out
heat_all_landsats['Coord'] = list(zip(heat_all_landsats['Long'], heat_all_landsats['Lat']))
print("done")
heat_all_landsats = heat_all_landsats.groupby('Coord').mean().reset_index()
print ('check')
    
remove_extremes = True
sd_cutoff = 4
if remove_extremes:
    heat_all_landsats = heat_all_landsats[np.abs(zscore(heat_all_landsats['ST_B10'])) <= sd_cutoff]

print(heat_all_landsats.head())

  0%|          | 0/5 [00:00<?, ?it/s]

done
check
                                      Coord    Year     ST_B10       Long  \
0   (-74.25532529025433, 40.50436242448614)  2021.0  24.463801 -74.255325   
1   (-74.25532529025433, 40.50463191907137)  2021.0  24.579444 -74.255325   
2   (-74.25532529025433, 40.50490141365661)  2021.0  24.674578 -74.255325   
3  (-74.25532529025433, 40.505170908241844)  2021.0  24.768574 -74.255325   
4  (-74.25532529025433, 40.505440402827084)  2021.0  24.954286 -74.255325   

         Lat  
0  40.504362  
1  40.504632  
2  40.504901  
3  40.505171  
4  40.505440  


In [5]:
#Convert to geodataframe
geometry = gpd.points_from_xy(heat_all_landsats.Long, heat_all_landsats.Lat)
crs = {'init': 'epsg:4326'} #http://www.spatialreference.org/ref/epsg/2263/
heat_all_landsats_gp = gpd.GeoDataFrame(heat_all_landsats, crs=crs, geometry=geometry)
heat_all_landsats_gp.head()

,Coord,Year,ST_B10,Long,Lat,geometry
0,"(-74.25532529025433, 40.50436242448614)",2021.0,24.463801,-74.255325,40.504362,POINT (-74.25533 40.50436)
1,"(-74.25532529025433, 40.50463191907137)",2021.0,24.579444,-74.255325,40.504632,POINT (-74.25533 40.50463)
2,"(-74.25532529025433, 40.50490141365661)",2021.0,24.674578,-74.255325,40.504901,POINT (-74.25533 40.50490)
3,"(-74.25532529025433, 40.505170908241844)",2021.0,24.768574,-74.255325,40.505171,POINT (-74.25533 40.50517)
4,"(-74.25532529025433, 40.505440402827084)",2021.0,24.954286,-74.255325,40.505440,POINT (-74.25533 40.50544)


In [6]:
# Optional - Decrease the data of heat_all_landsats_gp
sub_size = int(heat_all_landsats_gp.shape[0]/2)
heat_all_landsats_sub = heat_all_landsats_gp.iloc[np.random.choice(np.arange(heat_all_landsats_gp.shape[0]), replace=False, size=sub_size),:]
heat_all_landsats_sub.shape

(569339, 6)

In [7]:
# Optional - increasing radius of heat datapoints (only for pavement edges)
# heat_all_landsats_gp['geometry'] = heat_all_landsats_gp.geometry.buffer(0.1)

In [8]:
# Optional (check type of file)
# type(heat_all_landsats_gp)

## Return temperature stats of planimetric feature of choice

In [9]:
files = ["parking_lot_df_50.geojson", "Cooling Tower.geojson", "Public Plazas.geojson", "Hydrography Structures.geojson", 
         "Railroad Structure.geojson", "Parks Properties Of The City.geojson", "Transportation Structures.geojson",
         "Swimming Pools.geojson", "Parking Lot.geojson", "Sidewalk.geojson", "Pavement_Edge.geojson", "Roadbed.geojson",
        "Building Footprints.geojson"]

file_of_interest = (input("Select any item in 'files' list"))

utils.TempWithin(r"data\GEOJSONS", file_of_interest, heat_all_landsats_gp)

Select any item in 'files' listSwimming Pools.geojson
NYC Average Temperature: 97.7 +/- 6.4 F
Average Temperature of Swimming Pools.geojson: 96.08333244575027 +/- 4.107336230186534 F


# Visualizing Average Temperatures of all Planimetric Features

In [15]:
import plotly.express as pe
plani_df = pd.read_csv(r"data\Data From Planimetrics Project.csv")
plani_df_new = plani_df.drop(plani_df.index[plani_df["Planimetric Feature"] == "NYC"])


nyccolorplot = []
for i in plani_df_new["Temperature"]:
    nyccolorplot.append("Average NYC Temperature")

plani_df_new["NYC Only"] = nyccolorplot
fig_plani_2 = pe.line(plani_df_new, x= "Planimetric Feature", y=[plani_df["Temperature"].values[0]]*13, color = "NYC Only", color_discrete_map={
                "Average NYC Temperature": "black"}, )


tempcolorplot = []
for i in plani_df_new["Temperature"]:
    if i < plani_df[plani_df["Planimetric Feature"] == "NYC"]["Temperature"].values[0]:
        tempcolorplot.append("Below Average NYC Temperature")
    else:
         tempcolorplot.append("Above Average NYC Temperature")

plani_df_new["Temperature Category"] = tempcolorplot



fig_plani_1 = pe.bar(plani_df_new, x="Planimetric Feature", y="Temperature", title="Average Temperatures of NYC Planimetric Features", color = "Temperature Category", color_discrete_map={
                "Above Average NYC Temperature": "red",
                "Below Average NYC Temperature": "blue"}, error_y="Standard Deviation ")

fig_plani = (go.Figure(data= fig_plani_2.data + fig_plani_1.data))
fig_plani.update_layout(yaxis_range=[75,107], showlegend = True)
fig_plani.update_yaxes(title = "Temperatures (Fahrenheit)")
fig_plani.update_xaxes(title = "Planimetric Feature")

fig_plani.add_hline(y=plani_df[plani_df["Planimetric Feature"] == "NYC"]["Temperature"].values[0], line_color="black", line_width=3)

# fig_plani.update_layout(legend=dict(
#     yanchor="top",
#     y=1.5,
#     xanchor="right",
#     x=1
# ))

fig_plani.show()

## Visualizing planimetric features on top of the NYC Heat map

In [16]:
#Planimetric Feature Data Load
cd_boundaries_filepath = r"data\GEOJSONS\parking_lot_df_50.geojson"
with open(cd_boundaries_filepath) as f:
    cd_json = json.load(f)
districts_of_interest = np.array([x['properties']['source_id'] for x in cd_json['features']])


In [17]:
sub_size = int(heat_all_landsats.shape[0]/2)
heat_all_landsats_small = heat_all_landsats.iloc[np.random.choice(np.arange(heat_all_landsats.shape[0]), replace=False, size=sub_size),:]
heat_all_landsats_small = heat_all_landsats_small.drop(columns=["Coord"])

In [20]:
# Convert Temperatures to F, ZScore
plot_zscore = True 

degrees='F'
if degrees == 'C':
    pass
elif degrees == 'F':
    heat_all_landsats_small['ST_B10'] = [(x * 9/5) + 32 for x in heat_all_landsats_small['ST_B10']]
else:
    raise Exception('Invalid Temperature Scale. Must be C or F.')

# ZScore if necessary
if plot_zscore:
    heat_all_landsats_small['ST_B10'] = zscore(heat_all_landsats_small['ST_B10'])

zmin = heat_all_landsats_small['ST_B10'].min()
zmax = heat_all_landsats_small['ST_B10'].max()
print(zmin, zmax)

-4.0540858949996945 4.0380301925897015


In [21]:
#Plot City-wide heatmap (with community/council district boundaries)
var_to_plot = 'ST_B10'
plot_district_outlines = True

fig = go.Figure()
fig.add_trace(go.Scattermapbox(lat=heat_all_landsats_small['Lat'], lon=heat_all_landsats_small['Long'], mode= 'markers',
                               marker=dict(color=heat_all_landsats_small[var_to_plot],size=5, colorscale='RdYlBu_r', cmin=zmin, cmax=zmax, opacity=1), hoverinfo='skip'))

colorbar_trace  = go.Scatter(x=[None], y=[None], mode='markers', hoverinfo='none',
                            marker=dict(colorscale='RdYlBu_r', showscale=True,
                                         #cmin=landsat_pixel_df[var_to_plot].min(), cmax=landsat_pixel_df[var_to_plot].max(),
                                         #colorbar=dict(thickness=20, tickvals=[landsat_pixel_df[var_to_plot].min(), landsat_pixel_df[var_to_plot].max()], ticktext=['Low', 'High'], outlinewidth=0)
                                         cmin=zmin, cmax=zmax,
                                         colorbar=dict(thickness=20, tickvals=[zmin,zmax], outlinewidth=0, tickfont=dict(size=20),
                                                       ticktext=[np.round(zmin,1),np.round(zmax,1)])))
                                                       #ticktext=['Low','High'])))
fig['layout']['showlegend'] = False
fig.add_trace(colorbar_trace)
print ("check")

if plot_district_outlines:
    for i, district in enumerate(districts_of_interest):
        district_polygons = cd_json['features'][np.argwhere(districts_of_interest == district)[0][0]]['geometry']['coordinates']
        for polygon in district_polygons:
            fig.add_trace(go.Scattermapbox(
                fill = "none",
                lon = [p[0] for p in polygon[0]],
                lat = [p[1] for p in polygon[0]],
                mode='lines',
                #line = dict(width=0.1, color='black'), 
                line = dict(width=0.5, color='black'), hoverinfo='skip', 
                name=None))

fig.update_layout(
    geo_scope='usa', title_text='LANDSAT 8 Surface Temperature for 05/01-09/01 Averaged from 2017-2021',
    mapbox = {
        'accesstoken': mapbox_accesstoken,
        'style': 'carto-positron', #carto-positron, carto-darkmatter, dark, satellite, satellite-streets, streets, outdoors, basic
        'center': {'lon': -73.95, 'lat': 40.7 },
        'zoom': 9.9},
    width=1500, height=1200, template='none')
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
# fig.show(renderer='notebook_connected', config={'scrollZoom': False})
# fig.show(renderer='png')
fig.write_html(r'Planimetric Map Overlays\50 Biggest Parking Lots Overlay.html') #, scale=5)

check
